# Use example of *hsc_to_lci*

Questions:

- What are the relevant rows for inputs and outputs?
- If each unit process is a dataset, we need to define a reference product for each unit process, how the unit processes are connected, as well as the amount of each unit process entering the next one
- We need a mapping file between simulation variables and ecoinvent datasets... this can be tricky. For example, by default we can define the "alfa-spodumene" corresponds to the "market for spodumene" in ecoinvent... however it can also be "spodumene production in AU"
- Are the name of the streams standard or should they be defined for every simulation?

In [1]:
import project_path
from hsc_to_lci.converter import Converter
#from hsc_to_lci import utils

#import pandas as pd
#import yaml
#import brightway2 as bw

In [2]:
METADATA_PATH = "meta_data.yaml"
converter_obj = Converter(metadata = METADATA_PATH)

Importing the ecoinvent database...
Getting activity data


100%|██████████| 21238/21238 [00:00<00:00, 55738.45it/s]


Adding exchange data to activities


100%|██████████| 674593/674593 [00:47<00:00, 14286.32it/s]


Filling out exchange data


100%|██████████| 21238/21238 [00:03<00:00, 6743.60it/s] 


Importing the biosphere database...


In [3]:
converter_obj.create_lci_database()

Format inventories to Brightway2 format
Extract process simulation data
Apply strategies:
... add technosphere/biosphere flow type
... change units to ecoinvent format
... convert process simulation units to ecoinvent units


Writing activities to SQLite3 database:


Linking datasets within the database and to ecoinvent and biosphere databases
Done!
Writing LCI database to Brightway2 and exporting inventories in Excel file...
10 datasets
37 exchanges
0 unlinked exchanges
  


0% [##########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/31/2023 16:44:46
  Finished: 10/31/2023 16:44:46
  Total time elapsed: 00:00:00
  CPU %: 97.70
  Memory %: 35.07
Created database: lithium simulation inventories


'Database created and inventories exported to: c:\\Users\\istrateir\\Research\\Projects\\hsc_to_lci\\examples\\lithium simulation inventories_31-10-2023.xlsx'

In [3]:
streams_data = converter_obj.get_input_output_streams_data()

Extract process simulation data
Apply strategies:
... add technosphere/biosphere flow type
... change units to ecoinvent format
... convert process simulation units to ecoinvent units


In [4]:
streams_data

,Stream Name,Amount,Unit,Stream type,LCI type
Unit Name,,,,,
AcidRoasting,Sulphuric Acid,17179.49615,kilogram,Input Streams,technosphere
AcidRoasting,EnergyAR,7543.252056,kilowatt hour,Input Streams,technosphere
Burner,Natural gas,605.227435,cubic meter,Input Streams,technosphere
Burner,Air,7101.328477,cubic meter,Input Streams,technosphere
Carbonation,Soda Ash,11800.0,kilogram,Input Streams,technosphere
Drying,EnergyD,237.074335,kilowatt hour,Input Streams,technosphere
Drying,H2O(g),0.293154,cubic meter,Output Streams,biosphere
Evaporation,EnergyV,2149.081133,kilowatt hour,Input Streams,technosphere
Evaporation,H2O(g),2.726735,cubic meter,Output Streams,biosphere


In [14]:
import wurst
import bw2io
from constructive_geometries import *
import copy

In [9]:
unit_processes = list(set(streams_data.index))
unit_processes

['Burner',
 'Drying',
 'AcidRoasting',
 'Leaching',
 'Rotary Decrepitation Kiln',
 'Carbonation',
 'Purification',
 'Final Cooling',
 'Evaporation']

In [16]:
geomatcher = Geomatcher() # Initialize the geomatcher object 
LOCATION = converter_obj.metadata['system description']["location"]

for up in unit_processes:
    for index, row in streams_data.loc[[up]].iterrows():
        if row["LCI type"] == "technosphere":
            print(up)
            exc_filter = {'name': converter_obj.simulation_lci_map[row["Stream Name"]]["Name"],
                          'product': converter_obj.simulation_lci_map[row["Stream Name"]]["Reference product"],
                          'unit': row['Unit']}
            print(exc_filter)
            # Get both "market group" and "market" activities:
            if 'market for' in exc_filter['name']:
                possible_datasets_group = list(wurst.transformations.geo.get_possibles(exc_filter, converter_obj.ecoinvent_db)) 

                exc_filter_market = copy.deepcopy(exc_filter)
                exc_filter_market.update({'name': exc_filter['name'].replace('market', 'market group')})
                possible_datasets_market = list(wurst.transformations.geo.get_possibles(exc_filter_market, converter_obj.ecoinvent_db))

                possible_datasets = possible_datasets_group + possible_datasets_market
            
            else:
                possible_datasets = list(wurst.transformations.geo.get_possibles(exc_filter, converter_obj.ecoinvent_db))

            # Check if there is an exact match for the location
            match_dataset = [ds for ds in possible_datasets if ds['location'] == LOCATION]
            
            # If there is no specific dataset for the location, search for the supraregional locations
            if len(match_dataset) == 0:
                loc_intersection = geomatcher.intersects(LOCATION, biggest_first=False)
                loc_intersection = [i[1] if type(i)==tuple else i for i in loc_intersection]
                # Inser RoW before GLO
                loc_intersection.insert(loc_intersection.index("GLO"), "RoW")

                for loc in loc_intersection:
                    match_dataset = [ds for ds in possible_datasets if ds['location'] == loc]
                    if len(match_dataset) > 0:
                        break

            match_dataset = match_dataset[0]
            print(match_dataset["name"], match_dataset["reference product"], match_dataset["location"])
        

Burner
{'name': 'market for natural gas, high pressure', 'product': 'natural gas, high pressure', 'unit': 'cubic meter'}
market for natural gas, high pressure natural gas, high pressure RoW
Burner
{'name': 'market for compressed air, 1000 kPa gauge', 'product': 'compressed air, 1000 kPa gauge', 'unit': 'kilogram'}


IndexError: list index out of range

In [5]:
inventories = converter_obj.format_inventories_for_bw()

Format inventories to Brightway2 format
Extract process simulation data
Apply strategies:
... add technosphere/biosphere flow type
... change units to ecoinvent format
... convert process simulation units to ecoinvent units
Linking datasets within the database and to ecoinvent and biosphere databases
Done!
